# Chapter 6: Moments
 
This Jupyter notebook is the Python equivalent of the R code in section 6.9 R, [Introduction to Probability, 1st Edition](https://www.crcpress.com/Introduction-to-Probability/Blitzstein-Hwang/p/book/9781466575578), Blitzstein & Hwang.

----

In [1]:
import numpy as np

## Functions

The MGF of an r.v. is a _function_. As an example of defining and working with functions in Python, let's use the $N(0, 1)$ MGF, which is given by $M(t) = e^{\frac{t^{2}}{2}}$. The code

In [2]:
def M(t):
    return np.exp(t**2/2)

print('calling function M with a single value: \nM(0) = {}\n'.format(M(0)))

print('calling M with a vector: \nM(np.arange(1,11))) = {}'.format(M(np.arange(1,11))))

calling function M with a single value: 
M(0) = 1.0

calling M with a vector: 
M(np.arange(1,11))) = [  1.64872127e+00   7.38905610e+00   9.00171313e+01   2.98095799e+03
   2.68337287e+05   6.56599691e+07   4.36731791e+10   7.89629602e+13
   3.88084696e+17   5.18470553e+21]


defines `M` to be this function. The `def` Python keyword says that we're [defining a function](https://docs.python.org/3/tutorial/controlflow.html#defining-functions). The function is named `M`, and it takes one variable `t` (called the argument or parameter of the function). The line declaring the function name and list of parameter(s) is terminated with a colon `:`, with the body of the function following on the next line after an indent. Note that a simple Python function will not be able to flexibly deal with both single or vector inputs, such as is possible with a function in R. However, since our function `M` relies on [`numpy.exp`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html), `M` can accept both a nested sequence of objects or `numpy` arrays as inputs, and return an single or tuple of `numpy` array as output.

Writing

    def M(x):
        return np.exp(x**2/2)

would define the same function `M`, except that now the argument is named `x`. Giving the arguments names is helpful for functions of more than one variable, since Python then saves us from having to remember the order in which to write the arguments, and allows us to assign default values. For example, the $N(\mu, \sigma^2)$ MGF is given by $g(t) = exp\left(\mu \, t + \frac{\sigma^2 \, t^2}{2} \right)$, which depends on $t$, $mu$, and $\sigma$. We can define this in Python by

In [3]:
def g(t, mean=0, sd=1):
    return np.exp(mean*t + (sd**2 * t**2)/2)

print('N(2, 3) MGF evaluated at 1 = {}'.format(g(1, 2, 3)))

N(2, 3) MGF evaluated at 1 = 665.1416330443618


What is `g(1, 2, 3)`? It's the $N(2, 3^{2})$ MGF evaluated at 1, but it may be hard to remember which argument is which, especially when working with many functions with many arguments over the course of many months. So we can also write `g(1, mean=2, sd=3)` or `g(1, sd=3, mean=2)`. Since the `mean` and `sd` function parameters have the form `parameter = expression`, the function is said to have "default parameter values."

Also, when defining `g` we specified default values of 0 for the mean and 1 for the standard deviation, so if we want the $N(0, 5^2)$ MGF evaluated at 3, we can use `g(3, sd=5)` as shorthand. It would be bad here to write `g(3, 5)`, since that is ambiguous about which argument is omitted; in fact, Python interprets this as `g(t3, mean=5)`.

In [4]:
print('g(1, mean=2, sd=3) = {}\t... explicitly using parameter names'.format(g(1, mean=2, sd=3)))

print('g(3, 5) = {}\t\t... which parameter is omitted?'.format(g(3, 5)))

print('g(3, mean=5) = {}\t... \'mean\' parameter was omitted'.format(g(3, mean=5)))

g(1, mean=2, sd=3) = 665.1416330443618	... explicitly using parameter names
g(3, 5) = 294267566.0415088		... which parameter is omitted?
g(3, mean=5) = 294267566.0415088	... 'mean' parameter was omitted


## Moments

LOTUS makes it easy to write down any moment of a continuous r.v. as an integral. The [`scipy.integrate`](https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html) module in SciPy can help us do the integral numerically, using the [`scipy.integrate.quad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad) function.

In [5]:
from scipy.integrate import quad

#print(quad.__doc__)

For example, let's approximate the 6<sup>th</sup> moment of a $N(0, 1)$ r.v. The code

In [6]:
from scipy.stats import norm

def g(x):
    return x**6 * norm.pdf(x)

y, abserr = quad(g, -np.inf, np.inf)
print('6th moment of N(0,1) = {}, with error = {}'.format(y, abserr))

6th moment of N(0,1) = 15.000000000000004, with error = 4.4229319318339374e-09


asks `quad` to compute $\int_{-\infty}^{\infty} g(x) \, dx$, where $g(x) = x^6 \, \phi(x)$ with $\phi$ the $N(0, 1)$ PDF. When we ran this, `quad` reported 15 (the correct answer, as we know from this chapter!) and that the absolute error was less than 4.423 $\times$ 10<sup>−9</sup>. 

However, all of the continuous distributions supported in [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/tutorial/stats/continuous.html#moments) have a `moment(n, loc=0, scale=1)` function that allows you to quickly and easily obtain the n<sup>th</sup> order non-central moment of the continuous distribution in question. For example:

In [7]:
from scipy.stats import moment

#print(moment.__doc__)

print('scipy.stats.norm.moment(n=6) = {}'.format(norm.moment(n=6)))

scipy.stats.norm.moment(n=6) = 15.00000000089533


Similarly, to check that the 2<sup>nd</sup> moment (and variance) of a $Unif(−1, 1)$ r.v. is 1/3, we can use `quad` again

In [8]:
from scipy.stats import uniform

def h(x):
    """ scipy.stats.uniform is constant between
            a = loc
            b = loc + scale
    """
    a = -1
    b = 1
    loc = a
    scale = b - loc
    return x**2 * uniform.pdf(x, loc=loc, scale=scale)

y, abserr = quad(h, -1, 1)
print('2nd moment of Unif(-1,1) = {}, with error = {}'.format(y, abserr))

2nd moment of Unif(-1,1) = 0.3333333333333333, with error = 3.700743415417188e-15


&#x2623; 6.9.1. Numerical integration runs into difficulties for some functions; as usual, checking answers in multiple ways is a good idea. Using `numpy.inf` for parameter `b` (the upper limit of integration) is preferred to using a large number as the upper limit when integrating up to $\infty$ (and likewise for a lower limit of `a = -numpy.inf` for $-\infty$. For example, on many systems `quad(norm.pdf, 0, 10**6)` reports 0.0 while `quad(norm.pdf, 0, numpy.inf)` reports the correct answer, 0.5.

Alternately, we can either use `moment(n=2, loc=-1, scale=2)` or just `var(loc=-1, scale=2)`, keeping in mind that `loc=-1` and `scale = interval length = 2`.

In [9]:
print('uniform.moment(n=2, loc=-1, scale=2) = {}'.format(uniform.moment(n=2, loc=-1, scale=2)))

print('uniform.var(loc=-1, scale=2) = {}'.format(uniform.var(loc=-1, scale=2)))

uniform.moment(n=2, loc=-1, scale=2) = 0.33333333333333326
uniform.var(loc=-1, scale=2) = 0.3333333333333333


For moments of a discrete r.v., we can use LOTUS and the `numpy.sum` function. For example, to find the 2<sup>nd</sup> moment of $X \sim Pois(7)$, we can use

In [10]:
from scipy.stats import poisson

def g(k):
    return k**2 * poisson.pmf(k, 7)

# we want to sum up to and including 100, so the upper limit is 100+1
ans = np.sum(g(np.arange(0, 100+1)))
print('2nd moment of Pois(7) = {}'.format(ans))

2nd moment of Pois(7) = 55.99999999999994


Here we summed up to 100 since it’s clear after getting a sense of the terms that the total contribution of all the terms after k = 100 is negligible (choosing an upper limit in this way is in contrast to how to use the integrate command in the continuous case). The result is extremely close to 56, which is comforting since $E(X^{2}) = Var(X) + (EX)^{2} = 7 + 49 = 56$.

But similar to continous r.v., the [discrete r.v. in `scipy.stats`](https://docs.scipy.org/doc/scipy/reference/tutorial/stats/discrete.html#moments) have a `moment` function as well.

In [11]:
print('poisson.moment(n=2, mu=7) = {}'.format(poisson.moment(n=2, mu=7)))

poisson.moment(n=2, mu=7) = 56


A sample moment can be found in easily using Numpy. If `x` is a vector of data, then [`numpy.mean(x)`](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.mean.html) gives its sample mean and, more generally, `numpy.mean(x**n)` gives the n<sup>th</sup> sample mean for any positive integer `n`. For example,

In [12]:
# seed the random number generator
np.random.seed(123)

x = norm.rvs(size=100)
print(np.mean(x**6))

17.0336574808


gives the 6<sup>th</sup> sample moment of 100 i.i.d.  $N(0, 1)$ r.v.s. How close is it to the true 6<sup>th</sup> moment? How close are other sample moments to the corresponding true moments?

The sample variance can also be found in easily with Numpy. If `x` is a vector of data, then using the `ddof` parameter (delta degrees of freedom) such as in [`numpy.var(x, ddof=1)`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.var.html) gives its sample variance. This returns `nan` (not a number) as well as issuing `RuntimeWarning: Degrees of freedom <= 0 for slice` (the divisor used in the calculation is `len(x) - ddof`) if `x` has length 1, since the $n − 1$ in the denominator is 0 in this case. It makes sense not to return a numerical value in this case, not only because of the definition but also because it would be insane to try to estimate the variability of a population if we only have one observation!

For a simple demonstration of using the sample mean and sample variance to estimate the true mean and true variance of a distribution, we generate 1000 times from a $N(0, 1)$ distribution and store the values in `z`. We then compute the sample mean and sample variance with `numpy.mean(z)` and `numpy.var(z, ddof=1)`.

In [13]:
np.random.seed(123)
z = norm.rvs(size=1000)

print('sample mean of z: {}'.format(np.mean(z)))
print('sample variance of z: {}'.format(np.var(z, ddof=1)))

with open('z.txt', 'w') as f:
    for e in z:
        f.write('{}\n'.format(e))

sample mean of z: -0.03956413608079184
sample variance of z: 1.0025782735213273


We find that `numpy.mean(z)` is close to 0 and `numpy.var(z, ddof=1)` is close to 1. You can try this out for a $N(\mu, \sigma^2)$ distribution (or other distribution) of your choosing; just remember that `numpy.norm.rvs` takes $\sigma$ as the `scale` parameter, and not $\sigma^2$!

The sample standard deviation of `x` can be found using [`numpy.std(x, ddof=1)`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html). This gives the same result as `numpy.sqrt(numpy.var(x, ddof=1))`.

In [14]:
print('np.std(z, ddof=1) = {}'.format(np.std(z, ddof=1)))

print('np.sqrt(np.var(z, ddof=1)) = {}'.format(np.sqrt(np.var(z, ddof=1))))

np.std(z, ddof=1) = 1.001288306893338
np.sqrt(np.var(z, ddof=1)) = 1.001288306893338


While the [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) module does have functions for [`skew`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html) and [`kurtosis`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html), both functions rely on the _population standard deviation_ (with $n$ rather than $n-1$ in the denominator).

However, we can easily define our own functions for _sample skewness_ and _sample kurtosis_ by using [`scipy.stats.moment`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.moment.html) and [`numpy.std(z, ddof=1)`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html).

In [15]:
def skew(x, use_sample_sd=True):
    """ Return the skew of x.
    
        Default is to use sample standard deviation on the denominator,
        yielding sample skew. 
        
        Specifying use_sample_sd=False is the same as using
        scipy.stats.skew(x).
    """
    ddof = 1 if use_sample_sd==True else 0
    return moment(x, 3) / np.std(x, ddof=ddof)**3

print('sample skew of x = {}'.format(skew(z)))

sample skew of x = -0.028996564222850806


In [16]:
def kurt(x, use_sample_sd=True):
    """ Return the excess kurtosis of x.
    
        Default is to use sample standard deviation on the denominator,
        yielding sample kurtosis. 
        
        Specifying use_sample_sd=False is the same as using
        scipy.stats.kurtosis(x).
    """
    ddof = 1 if use_sample_sd==True else 0
    return moment(x, 4) / np.std(x, ddof=ddof)**4 - 3.0

print('sample kurtosis of x = {}'.format(kurt(z)))

sample kurtosis of x = -0.03138467715866655


## Medians and modes

To find the median of a continuous r.v. with CDF $F$, we need to solve the equation $F(x) = 1/2$ for $x$, which is equivalent to finding the root (zero) of the function $g$ given by $g(x) = F(x) − 1/2$. This can be done using uniroot in R. For example, let's find the median of the Expo(1) distribution. The code

asks R to find a root of the desired function between 0 and 1. This returns an answer very close to the true answer of log(2) ≈ 0.693. Of course, in this case we can solve 1 − e −x = 1/2 directly without having to use numerical methods.

&#x2623; 6.9.2. The uniroot command is useful but it only attempts to find one root (as the name suggests), and there is no guarantee that it will find a root.

An easier way to find the median of the Expo(1) in R is to use qexp(1/2). The function qexp is the quantile function of the Expo(1) distribution, which means that qexp(p) is the value of x such that P(X ≤ x) = p for X ∼ Expo(1).

For finding the mode of a continuous distribution, we can use the optimize function in R. For example, let’s find the mode of the Gamma(6, 1) distribution, which is an important distribution that we will introduce in the next chapter. Its PDF is proportional to x 5 e −x. Using calculus, we can find that the mode is at x = 5. Using R, we can find that the mode is very close to x = 5 as follows.

If we had wanted to minimize instead of maximize, we could have put maximum=FALSE.

Next, let’s do a discrete example of median and mode. An interesting fact about the Bin(n, p) distribution is that if the mean np is an integer, then the median and mode are also np (even if the distribution is very skewed). To check this fact about the median for the Bin(50, 0.2) distribution, we can use the following code.

The which.max function finds the location of the maximum of a vector, giving the index of the first occurrence of a maximum. Since TRUE is encoded as 1 and FALSE is encoded as 0, the first maximum in pbinom(0:n,n,p)>=0.5 is at the first value for which the CDF is at least 0.5. The output of the above code is 11, but we must be careful to avoid an off-by-one error: the index 11 corresponds to the median being 10, since we started evaluating the CDF at 0. Similarly, which.max(dbinom(0:n,n,p)) returns 11, showing that the mode is at 10.

The sample median of a vector x of data can be found using median(x). But mode(x) does not give the sample mode of x (rather, it gives information about what type of object x is). To find the sample mode (or sample modes, in case there are ties), we can use the following function.

## Dice simulation

In the starred Section 6.7, we showed that in rolling 6 fair dice, the probability of a total of 18 is 3431/6 6. But the proof was complicated. If we only need an approximate answer, simulation is a much easier approach. And we already know how to do it! Here is the code for a million repetitions:

In our simulation this yielded 0.07346, which is very close to 3431/6 6 ≈ 0.07354.

----

&copy; Blitzstein, Joseph K.; Hwang, Jessica. Introduction to Probability (Chapman & Hall/CRC Texts in Statistical Science).